In [21]:
import pandas as pd
import psycopg2

In [22]:
conn = psycopg2.connect("host=127.0.0.1 user=postgres password=zoot")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [23]:
cur.execute("CREATE DATABASE world_cup")

In [24]:
cur.close()
conn.close()

In [25]:
conn = psycopg2.connect("host=127.0.0.1 user=postgres password=zoot dbname=world_cup")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [26]:
cur.execute("CREATE TABLE IF NOT EXISTS attendance( \
    year_and_host varchar,\
    total_attendance varchar, \
    matches varchar, \
    highest_attendance varchar, \
    vs varchar, \
    stage varchar)")

cur.execute("CREATE TABLE IF NOT EXISTS award( \
    year_and_host varchar, \
    golden_boot varchar, \
    goal int)")

cur.execute("CREATE TABLE IF NOT EXISTS finals( \
    year_and_host varchar,\
    champion varchar, \
    score_final varchar, \
    runnerup varchar, \
    third varchar, \
    score_3rdplace varchar, \
    fourth varchar)")

In [27]:
df_attendance = pd.read_csv("./data/FIFA World Cup Attendance.csv")
df_attendance["year_and_hosts"] = df_attendance[['Year', 'Hosts']].agg(' '.join, axis=1)
df_attendance.drop(columns=['Year', 'Hosts', 'Unnamed: 0', 'Average_Attendance', 'Venue'], inplace=True)

In [28]:
df_award = pd.read_csv('./data/FIFA World Cup Award.csv')
df_award_clean = df_award[['World _Cup', 'Golden _Boot', 'Goals']]

In [29]:
df_finals = pd.read_csv('./data/List of FIFA World Cup finals.csv')
df_finals['Year'] = df_finals['Year'].astype(str)
df_finals['year_and_host'] = df_finals[['Year', 'Host']].agg(' '.join, axis=1)
df_finals.drop(columns=['No. _ofteams', 'Unnamed: 0', 'Year', 'Host'], inplace=True)

In [30]:
attendance_table_insert = ("""INSERT INTO attendance(
    total_attendance,
    matches,
    highest_attendance,
    vs,
    year_and_host)
    VALUES (%s, %s, %s, %s, %s)
""")

award_table_insert = ("""
INSERT INTO award(
    year_and_host,
    golden_boot,
    goal
)
VALUES(%s, %s, %s)
""")

finals_table_insert = ("""
INSERT INTO finals(
    champion,
    score_final,
    runnerup,
    third,
    score_3rdplace,
    fourth,
    year_and_host
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
""")

In [31]:
for i, row in df_attendance.iterrows():
    cur.execute(attendance_table_insert, row)

In [32]:
for i, row in df_award_clean.iterrows():
    cur.execute(award_table_insert, row)

In [33]:
for i, row in df_finals.iterrows():
    cur.execute(finals_table_insert, row)

In [34]:
cur.close()
conn.close()